<a href="https://colab.research.google.com/github/sugiyama404/ReinfoceLearningForTrading/blob/main/Ape-X/Ape_X_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from google.colab import drive
import copy

from datetime import datetime
from matplotlib import pyplot as plt
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, ReLU, Input, Lambda
from tensorflow.keras.optimizers import Adam
from statistics import mean
import tensorflow.keras as keras
from tensorflow.keras.losses import Huber
from tensorflow.keras import backend as K

from sklearn.preprocessing import StandardScaler
from concurrent.futures import ThreadPoolExecutor, as_completed
from queue import Queue
from time import sleep
import random, string

from tensorflow.keras.utils import Progbar

mode = 'train'
name = 'apex'

level = 1
if level == 2:
    name += name + 'lv2'

drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
nov_path = '/content/drive/My Drive/' + nov_dir + f'sp500_{mode}.csv'

exp_dir = 'Colab Notebooks/workspace/export/'
mdl_dir = '/content/drive/My Drive/' + exp_dir + 'models'
csv_path = '/content/drive/My Drive/' + exp_dir + f'csv_data/{name}_{mode}.csv'

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

Mounted at /content/drive/


In [2]:
class Environment:
    def __init__(self, df, initial_money=100000, mode = 'test', commission = 0):

        self.df = df.dropna().reset_index()

        self.df_total_steps  = len(self.df)-1
        self.initial_money   = initial_money
        self.mode            = mode
        self.commission      = commission
        self.trade_time      = None
        self.trade_win       = None
        self.brfore_buy_cash = None
        self.action_space    = np.array([0, 1, 2]) # buy,hold,sell
        self.hold_a_position = None
        self.now_price       = None
        self.cash_in_hand    = None
        self.sell_price      = None
        self.buy_price       = None

        self.reset()
        
    def reset(self):

        self.trade_time      = 0
        self.trade_win       = 0
        self.brfore_buy_cash = 0
        self.end_step        = self.df_total_steps
        self.now_step        = 0
        self.hold_a_position = 0.0
        self.now_price       = self.df.loc[self.now_step, 'SP500']
        self.cash_in_hand    = self.initial_money
        self.sell_price      = 0
        self.buy_price       = 0

        return self._get_now_state()

    def step(self, action):

        self.now_step += 1
        self.now_price = self.df.loc[self.now_step, 'SP500']
 
        done = (self.end_step == self.now_step)

        self.sell_price = 0
        self._trade(action,done)
        reward = 0
        if (self.sell_price > 0) and (self.buy_price > 0) and ((self.sell_price - self.buy_price) != 0):
            reward = (self.sell_price - self.buy_price) / self.buy_price
            self.buy_price = 0
        cur_revenue = self._get_revenue()
 
        info = { 'cur_revenue' : cur_revenue , 'trade_time' : self.trade_time, 'trade_win' : self.trade_win }

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        state = np.empty(3)
        state[0] = self.hold_a_position
        state[1] = self.now_price
        state[2] = self.cash_in_hand
        return state

    def _get_revenue(self): 
        return self.hold_a_position * self.now_price + self.cash_in_hand

    def _trade(self, action,lastorder = False):
        if lastorder:
            self.cash_in_hand += self.now_price * self.hold_a_position
            self.hold_a_position = 0
            self.trade_time += 1
            if self.cash_in_hand > self.brfore_buy_cash:
                self.trade_win += 1
        else:
            if self.action_space[0] == action: # buy
                if self.hold_a_position == 0:
                    buy_flag = True
                    self.brfore_buy_cash = copy.copy(self.cash_in_hand)
                    while buy_flag:
                        if self.cash_in_hand > self.now_price:
                            self.hold_a_position += 1
                            self.buy_price += self.now_price
                            self.cash_in_hand -= self.now_price + self.commission * self.now_price
                        else:
                            buy_flag = False
            if self.action_space[2] == action: # sell
                if self.hold_a_position != 0:
                    self.sell_price += self.now_price * self.hold_a_position
                    self.cash_in_hand += self.now_price * self.hold_a_position - self.commission * self.now_price * self.hold_a_position
                    self.hold_a_position = 0
                    self.trade_time += 1
                    if self.cash_in_hand > self.brfore_buy_cash:
                        self.trade_win += 1

In [3]:
class Brain:
    def __init__(self):

        learning_rate = 0.00001
        neurons_per_layer = 24

        input = Input(shape=(3,))
        common = Dense(neurons_per_layer*2, activation='relu')(input)
        common = Dense(neurons_per_layer*4, activation='relu')(common)

        common = Dense(4, activation='linear')(common)
        output = Lambda(lambda a: K.expand_dims(a[:, 0], -1) + a[:, 1:] - 0.0*K.mean(a[:, 1:], keepdims=True),output_shape=(3,))(common)

        model = keras.Model(inputs=input, outputs=output)

        optimizer = Adam(learning_rate=learning_rate, clipvalue=40)
        model.compile(loss=Huber(), optimizer=optimizer)
        model.summary()
        self.model = model

In [4]:
class Learner:
    def __init__(self, model1, model2, memory):

        self.model1 = model1
        self.model2 = model2
        self.memory = memory
        self.gamma  = 0.99

    def learn(self, queue):
        while True:
            val = queue.get()
            if (val == 'done_prosess'):
                break
            s_flag = 11 if np.random.random() <= 0.5 else 22

            states, next_states, actions, rewards, done = val['state'], val['next_state'], val['act'], val['reward'], val['done']

            if s_flag == 11:
                q = self.model1.predict(states)  
                next_q = self.model2.predict(next_states)
                target = np.copy(q)

                target[:, actions] = rewards + (1 - done) * self.gamma * np.amax(next_q, axis=1)
                self.model1.train_on_batch(states, target)
            else:
                q = self.model2.predict(states)  
                next_q = self.model1.predict(next_states)
                target = np.copy(q)

                target[:, actions] = rewards + (1 - done) * self.gamma * np.amax(next_q, axis=1)
                self.model2.train_on_batch(states, target)
            self._tderror()
            self.memory.placement(self.model1, self.model2)
            queue.task_done()

    def _predict(self, state, s_flag = 12):
        values = None
        q1 = self.model1.predict(state)
        q2 = self.model2.predict(state)
        if s_flag == 12:
            values = q1 + q2
        elif s_flag == 11:
            values = q1 + q1
        else:
            values = q2 + q2
        return values

    def _tderror(self):
        states, next_states, acts, rewards, done, tderrors = self.memory.findall()
        
        n2_rewards = rewards[2:-1]
        next_rewards = rewards[1:-2]
        n3_action = acts[3:]
        n3_next_states = next_states[3:]

        states = states[:-3]
        next_states = next_states[:-3]
        acts = acts[:-3]
        rewards = rewards[:-3]
        done = done[:-3]

        next_action = np.argmax(self._predict(n3_next_states), axis = 1)
        next_action = next_action.reshape(-1)

        acts = acts.reshape(-1)

        acts_one = np.eye(3)[acts]
        next_action_one = np.eye(3)[next_action]
        n3_q = np.max(self._predict(n3_next_states) * next_action_one, axis = 1)

        target = rewards + self.gamma * next_rewards + (self.gamma ** 2) * n2_rewards + (self.gamma ** 3) * n3_q
        tderror = target - np.max(self._predict(states) * acts_one, axis = 1)
        self.memory.replace_tderror(tderror)

In [5]:
class Memory:
    def __init__(self, model1, model2, max_size=500, batch_size=32):

        self.cntr = 0
        self.size = 0
        self.max_size = max_size
        self.batch_size = batch_size
        self.model1 = model1
        self.model2 = model2
        self.code = ''
        self.states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.next_states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.acts_memory = np.zeros(self.max_size, dtype=np.uint8)
        self.rewards_memory = np.zeros(self.max_size, dtype=np.float32)
        self.done_memory = np.zeros(self.max_size, dtype=np.uint8)
        self.tderrors_memory = np.zeros(self.max_size, dtype=np.float32)

    def store_transition(self, state, act, reward, next_state, done):
        self.states_memory[self.cntr] = state
        self.next_states_memory[self.cntr] = next_state
        self.acts_memory[self.cntr] = act
        self.rewards_memory[self.cntr] = reward
        self.done_memory[self.cntr] = done
        self.cntr = (self.cntr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def random_sampling(self):
        mb_index = np.random.choice(self.size, self.batch_size, replace=False)
        key = ['state','next_state','act','reward','done']
        value = [self.states_memory[mb_index],self.next_states_memory[mb_index],
                 self.acts_memory[mb_index],self.rewards_memory[mb_index],
                 self.done_memory[mb_index]]
        dict1=dict(zip(key,value))
        return dict1

    def findall(self):
        return self.states_memory,self.next_states_memory,self.acts_memory,self.rewards_memory,self.done_memory,self.tderrors_memory

    def replace_tderror(self, val):
        np.put(self.tderrors_memory, range(val.shape[0]), val)

    def placement(self, model1, model2):
        self.model1.set_weights(model1.get_weights())
        self.model2.set_weights(model2.get_weights())
        self.code = self._make_random_code()

    def integration(self, model1, model2, code):
        if self.code == code:
            return
        model1.set_weights(self.model1.get_weights())
        model2.set_weights(self.model2.get_weights()) 
        code = self.code

    def _make_random_code(self, n=10):
       return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

In [6]:
class Agent:
    def __init__(self, model1, model2, memory, epsilon, batch_size=32):
        self.model1 = model1
        self.model2 = model2
        self.memory = memory
        self.gamma = 0.99
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.epsilon_min = 0.01
        self.code = 'init'

    def update_replay_memory(self, state, action, reward, next_state, done):
        self.memory.store_transition(state, action, reward, next_state, done)

    def act(self, state, s_flag=12):
        ran_num = np.random.rand()
        if (ran_num <= self.epsilon) and (ran_num > self.epsilon_min):
            return np.random.choice(3)
        act_values = self._predict(state,s_flag)
        return np.argmax(act_values)

    def _predict(self, state, s_flag = 12):
        values = None
        q1 = self.model1.predict(state)
        q2 = self.model2.predict(state)
        if s_flag == 12:
            values = q1 + q2
        elif s_flag == 11:
            values = q1 + q1
        else:
            values = q2 + q2
        return values

    def replay(self):
        m_batch = self.memory.random_sampling()
        return m_batch

    def pioritized_experience_replay(self):

        sum_ab_tderror = self._absolute_tderror()
        td_list = np.random.uniform(0, sum_ab_tderror, self.batch_size)
        td_list = np.sort(td_list)

        num_np = np.array([], dtype=np.int16)
        i, sum_ab_tderror_tmp = 0, 0
        states, next_states, actions, rewards, done, tderror = self.memory.findall()
        for item in td_list:
            while sum_ab_tderror_tmp < item:
                sum_ab_tderror_tmp += abs(tderror[i]) + 0.0001
                i += 1
            num_np = np.append(num_np, i)

        key = ['state','next_state','act','reward','done']
        value = [states[num_np],next_states[num_np],
                 actions[num_np],rewards[num_np],
                 done[num_np]]
        dict1=dict(zip(key,value))
        return dict1

    def tderror(self):
        states, next_states, acts, rewards, done, tderrors = self.memory.findall()
        
        n2_rewards = rewards[2:-1]
        next_rewards = rewards[1:-2]
        n3_action = acts[3:]
        n3_next_states = next_states[3:]

        states = states[:-3]
        next_states = next_states[:-3]
        acts = acts[:-3]
        rewards = rewards[:-3]
        done = done[:-3]

        #next_action = np.argmax(self._predict(n3_next_states), axis = 1)
        #next_action = next_action.reshape(-1)
        next_action = n3_action.reshape(-1)
        acts = acts.reshape(-1)

        next_action_one = np.eye(3)[next_action]
        acts_one = np.eye(3)[acts]
        n3_q = np.max(self._predict(n3_next_states) * next_action_one, axis = 1)

        target = rewards + self.gamma * next_rewards + (self.gamma ** 2) * n2_rewards + (self.gamma ** 3) * n3_q
        tderror = target - np.max(self._predict(states) * acts_one, axis = 1)
        self.memory.replace_tderror(tderror)

    def _absolute_tderror(self):
        absolute_tderror = 0
        tderror = self.memory.tderrors_memory
        for i in range(0, (len(tderror)-1)):
            absolute_tderror += abs(tderror[i]) + 0.0001
        return absolute_tderror

    def integration(self):
        self.memory.integration(self.model1, self.model2, self.code)

    def load(self, name, name2):
        self.model1.load_weights(name)
        self.model2.load_weights(name2)

    def save(self, name, name2):
        self.model1.save_weights(name)
        self.model2.save_weights(name2)

In [7]:
class Main:
    def __init__(self, env, agent, mdl_dir, name, i, term, episodes_times = 200, mode = 'test'):
        self.env            = env
        self.agent          = agent
        self.mdl_dir        = mdl_dir
        self.scaler         = self._standard_scaler(self.env)
        self.episodes_times = episodes_times
        self.mode           = mode
        self.name           = name
        self.agent_num      = i
        self.term           = term

        with open(csv_path, 'w') as f:
            row = 'FixedProfit,TradeTimes,TradeWin'
            print(row, file=f)

        if self.mode == 'test':
            self._load()
            self.agent.epsilon = 0.01

    def play_game(self, q):

        total_reward = [1000000]

        for episode in range(self.episodes_times):

            if (episode % 10 == 0):
                metrics_names = ['FixedProfit','TradeTimes','TradeWin']
                pb_i = Progbar(10, stateful_metrics=metrics_names)
                p_mean,trade_time,win_time = np.array([]),np.array([]),np.array([])

            state = self.env.reset()
            state = self.scaler.transform([state])
            done  = False
        
            while not done:
                action = self.agent.act(state)
                next_state, reward, done, info = self.env.step(action)
                next_state = self.scaler.transform([next_state])
                reward = self._reward_clipping(reward)

                if self.mode == 'train':
                    self.agent.update_replay_memory(state, action, reward, next_state, done)
                    if (self.agent.memory.size > self.agent.memory.batch_size) and (self.agent.memory.cntr % 100 == 0):
                        self.agent.tderror()
                        m_batch = None
                        if mean(total_reward) < 1050000:
                            m_batch = self.agent.replay()
                        else:
                            m_batch = self.agent.pioritized_experience_replay()
                        q.put(m_batch)
                    self.agent.integration()

                state = next_state

            if self.mode == 'train':
                self.agent.integration()
                total_reward.append(info['cur_revenue'])

            p_mean,trade_time,win_time = np.append(p_mean,info['cur_revenue']),np.append(trade_time,info['trade_time']),np.append(win_time,info['trade_win'])
            values=[('FixedProfit',int(np.mean(p_mean))), ('TradeTimes',int(np.mean(trade_time))), ('TradeWin',int(np.mean(win_time)))]
            pb_i.add(1, values=values)
            with open(csv_path, 'a') as f:
                row = str(info['cur_revenue']) + ',' + str(info['trade_time']) + ',' + str(info['trade_win'])
                print(row, file=f)

        if self.mode == 'train':
            self._save()
            self.term[self.agent_num] = True
            if self.term.all():
                q.put("done_prosess")

    def _standard_scaler(self, env):
        states = []
        for _ in range(env.df_total_steps):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                break
        
        scaler = StandardScaler()
        scaler.fit(states)
        return scaler

    def _reward_clipping(self, val):
        if val > 0:
            return 1
        elif val == 0:
            return 0
        else:
            return -1

    def _load(self):
        with open('{}/{}_{}.pkl'.format(self.mdl_dir, self.name, str(self.agent_num)), 'rb') as f:
            self.scaler = pickle.load(f)
        self.agent.load('{}/{}_1.h5'.format(self.mdl_dir, self.name), '{}/{}_2.h5'.format(self.mdl_dir, self.name))

    def _save(self):
        with open('{}/{}_{}.pkl'.format(self.mdl_dir, self.name, str(self.agent_num)), 'wb') as f:
            pickle.dump(self.scaler, f)
        self.agent.save('{}/{}_1.h5'.format(self.mdl_dir, self.name), '{}/{}_2.h5'.format(self.mdl_dir, self.name))

In [8]:
initial_money=1000000
episodes_times = 150
commission = 0 if level == 1 else 0.002
batch_size = 32
max_size = 1000

brain1 = Brain()
model1 = brain1.model
brain2 = Brain()
model2 = brain2.model

memory = Memory(clone_model(model1), clone_model(model2), max_size, batch_size)
learner = Learner(model1, model2, memory)

thread_num = 4
epsilon_list = np.array([])
epsilon = 0.4
alfa = 7
for i in range(thread_num):
    num = 0.0
    if (not (thread_num - 1) == 0):
        num = epsilon ** (1 + i * alfa / (thread_num - 1))
    else:
        num = epsilon
    epsilon_list = np.append(epsilon_list, num)

workers = []
term = np.full((thread_num,), False)
for i in range(thread_num):
    env = Environment(df, initial_money=initial_money, mode = mode, commission = commission)
    epsilon = epsilon_list[i]
    agent = Agent(clone_model(model1), clone_model(model2), memory, epsilon, batch_size)
    main  = Main(env, agent, mdl_dir, name, i, term, episodes_times, mode)
    workers.append(main)

data = []
q = Queue()
with ThreadPoolExecutor() as executor:
    for i, w in enumerate(workers):
        print(f'start worker_{i}')
        def job1(): return w.play_game(q,)
        data.append(executor.submit(job1))

    if mode == 'train':
        print('start learner')
        def job2(): return learner.learn(q,)
        data.append(executor.submit(job2))
    for future in as_completed(data):
        print(future.result())
    executor.shutdown()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 48)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 96)                4704      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 388       
_________________________________________________________________
lambda (Lambda)              (None, 3)                 0         
Total params: 5,284
Trainable params: 5,284
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape 